### **Song Sage - Personalised Melodies**


## *Dependancies*




In [1]:
!pip install --upgrade openai
!pip install spotipy

from google.colab import drive
drive.mount('/content/drive')

# Initialize OpenAI client
from google.colab import userdata

from spotipy.oauth2 import SpotifyClientCredentials
import json
import time

import spotipy
import os

from openai import OpenAI, File

# Check the version of openai
!pip show openai | grep Version

# Define a function to display JSON nicely
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 6.6 MB/s eta 0:00:00
Mounted at /content/drive
Version: 1.25.0


In [2]:
# Initialize OpenAI client
client = OpenAI(api_key = userdata.get('API_KEY'))

assistant = client.beta.assistants.create(
    name="Song Sage",
    instructions="You are a Song aficionado tasked with extracting information specifically related to songs from the uploaded file. Look for mentions of song titles, artists, albums, and track details such as track URI, track name, artist URI, artist name, album URI, and album name. Exclude any content that references non-song topics or subjects outside of the songs world. Prioritize information that directly contributes to discussions, recommendations, analyses, summaries, or insights about songs content. Ensure that all retrieved data pertains solely to the realm of songs without referencing external sources or contexts.",
    model="gpt-3.5-turbo"
)

## *File Search - ChatBot*

In [3]:
# Initialize OpenAI client
client = OpenAI(api_key = userdata.get('API_KEY'))

assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tools=[{"type": "file_search"}],
)

# Create a vector store called "Song Sage"
vector_store = client.beta.vector_stores.create(name="Song Sage")

# Ready the files for upload to OpenAI
file_paths = ["/content/drive/MyDrive/GAI/songs.txt"]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("/content/drive/MyDrive/GAI/songs.txt", "rb"), purpose="assistants"
)

# Display the initial prompt
print("Song Sage: Hello! How can I assist you today with the Song-related questions or discussions you may have?")

# Main conversation loop
while True:
    user_input = input("You: ")

    # Exit if the user wants to close the Song Sage chatbot
    if user_input.lower() == "close":
        print("Closing Song Sage. Goodbye!")
        break

    # Create a thread with the user's question and attach the file
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                "content": user_input,
                "attachments": [
                    {"file_id": message_file.id, "tools": [{"type": "file_search"}]}
                ],
            }
        ]
    )

    # Retrieve the response from the assistant
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id
    )

    # Print the response
    messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
    if messages:
        print("Song Sage:", messages[-1].content[0].text.value)
    else:
        print("Song Sage: I'm sorry, I couldn't find a response to your question from the uploaded files.")


Song Sage: Hello! How can I assist you today with the Song-related questions or discussions you may have?
You: Hi
Song Sage: Hello! How can I assist you today with the additional files you uploaded?
You: Who is the artist of the song "Heaven Is A Place On Earth"?
Song Sage: The artist of the song "Heaven Is A Place On Earth" is Belinda Carlisle【4:0†source】.
You: Find me the track "Juke Box Hero" by Foreigner.
Song Sage: The track "Juke Box Hero" by Foreigner can be found on the album "4"【4:0†source】.
You: What is the Track URI for the song "Deep End"?
Song Sage: The Track URI for the song "Deep End" is spotify:track:1PZ5mBtpRXZj8aMgQQYcro【4:0†source】.
You: close
Closing Song Sage. Goodbye!


## *Function Calling - ChatBot*

In [4]:
# Define the function to recommend songs
def recommend_songs(language=None, year=None, artist=None):

    client_id = 'e6318e8f86084899a75a3c857c8b648b'
    client_secret = '3dc006d56e924a5c94759578636e9589'

    # Authenticate with Spotify API
    auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(auth_manager=auth_manager)

    # Construct the query based on user-provided parameters
    query = ""
    if language:
        query += f" lang:{language}"
    if year:
        query += f" year:{year}"
    if artist:
        query += f" artist:{artist}"

    # Search for songs based on the constructed query
    results = sp.search(q=query, type='track', limit=10)

    # Extract song recommendations
    recommended_songs = []
    for track in results['tracks']['items']:
        recommended_songs.append({
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'spotify_link': track['external_urls']['spotify']
        })

    return recommended_songs


    print(recommended_songs = [f"{song['name']}: {song['artist']} ({song['spotify_link']})" for song in recommended_songs])
    recommended_songs_str = ", ".join(song_titles)
    print(recommended_songs_str)



def get_outputs_for_tool_call(tool_call):
    # Parse the arguments of the tool call
    arguments = json.loads(tool_call.function.arguments)

    # Extract optional parameters: year, language, artist
    year = arguments.get("year")
    language = arguments.get("language")
    artist = arguments.get("artist")

    # Call the recommend_songs function with the extracted parameters
    recommended_songs = recommend_songs(year=year, artist=artist, language=language )
    # print(recommended_songs)


    # Construct a string containing the names of the recommended songs
    if recommended_songs:
      song_titles = [f"{song['name']}: {song['spotify_link']}" for song in recommended_songs]
      recommended_songs_str = ", ".join(song_titles)
    else:
      recommended_songs_str = "No Song Found"

    # Return the tool call ID along with the output
    return {
        "tool_call_id": tool_call.id,
        "output": recommended_songs_str
    }


# Define the tools list
tools_list = [
    {
        "type": "function",
        "function": {
            "name": "recommend_songs",
            "description": "Recommend songs based on user-provided criteria such as language, year, and artist.",
            "parameters": {
                "type": "object",
                "properties": {
                    "language": {
                        "type": "string",
                        "description": "The preferred language for song recommendations."
                    },
                    "year": {
                        "type": "string",
                        "description": "The year of the songs to be recommended."
                    },
                    "artist": {
                        "type": "string",
                        "description": "The preferred artist for song recommendations."
                    }
                }
            }
        }
    }
]

# Create a Song Sage with the song recommendation tool
assistant = client.beta.assistants.update(
    assistant_id = assistant.id,
    instructions="You are a Song aficionado with vast knowledge of various song languages, artist, year, and cultural significance.Search the provided content specifically related to songs. Look for mentions of song titles, characters, creators, studios, genres, themes, or cultural references associated with songs from the Spotify API. Exclude any content that references non-songs topics or subjects outside of the songs world. Prioritize information that directly contributes to discussions, recommendations, analyses, summaries, or insights about songs content. Ensure that all retrieved data pertains solely to the realm of songs without referencing external sources or contexts.",
    tools=tools_list
)

def get_response(user_input):
    # Create a thread and attach the file to the message
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                "content": user_input
            }
        ]
    )

    # Use the create and poll SDK helper to create a run and poll the status of the run until it's in a terminal state.
    run = client.beta.threads.runs.create(
        thread_id=thread.id, assistant_id=assistant.id,
        #instructions="You are a Song aficionado tasked with extracting information specifically related to songs from File search, GPT and API call. Look for mentions of song titles, artists, albums, and track details such as track URI, track name, artist URI, artist name, album URI, and album name. Exclude any content that references non-song topics or subjects outside of the songs world. Prioritize information that directly contributes to discussions, recommendations, analyses, summaries, or insights about songs content. Ensure that all retrieved data pertains solely to the realm of songs without referencing external sources or contexts."
    )

    status = ""
    attempts = 0
    MAX_ATTEMPTS = 3
    while attempts < MAX_ATTEMPTS:
        attempts += 1
        latest_run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        status = latest_run.status
        # print(status)
        if status == "completed":
            break
        elif status == "requires_action":
            tool_calls = latest_run.required_action.submit_tool_outputs.tool_calls
            tool_outputs = map(get_outputs_for_tool_call, tool_calls)
            tool_outputs = list(tool_outputs)
            run = client.beta.threads.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)
            # print(status)
            time.sleep(1)
        elif status == "failed":
            return "Failed to retrieve response"
        time.sleep(1)

    # Retrieve the assistant's response
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for msg in messages.data:
        if msg.role == "assistant":
            return msg.content[0].text.value

    return "Failed to retrieve response"

In [6]:
# Main conversation loop
print("SongSage: Hello! How can I assist you today with song recommendations?")
while True:
    user_input = input("You: ")

    # Exit if the user wants to close the chatbot
    if user_input.lower() == "close":
        print("SongSage: Goodbye!")
        break

    response = get_response(user_input)

    # Print the response
    print("SongSage:", response)

SongSage: Hello! How can I assist you today with song recommendations?
You: What are the 2 top songs by Ed Sheeran on Spotify year of 2024?
SongSage: The top 2 songs by Ed Sheeran on Spotify in the year 2024 are: 

1. "Perfect" - [Listen here](https://open.spotify.com/track/17IOrfDXBnYgnUotW0AKwt)
2. "Castle on the Hill" - [Listen here](https://open.spotify.com/track/1fyk9bDbUs1TMgVvr5PJSk)
You: close
SongSage: Goodbye!
